# Simple Document Encryption and Verification using Asymmetric Keys with OpenSSL

### Usecase:
* Mr.A ต้องการส่งข้อความเข้ารหัสไปให้ Mr.B
* การสื่อสารระหว่าง Mr.A และ Mr.B ต้องเข้ารหัสและถอดรหัสด้วย Asymmetric Keys
* Mr.B ต้องการ verify ผู้ส่งและข้อความว่าถูกต้องหรือไม่ด้วย MD5 Digital Signature
* แลกเปลี่ยน Key และ Verify กันสองคนโดยไม่มี Trusted party และรับส่งผ่านทาง file



### Mr.B

Mr.B สร้าง private key สำหรับการสร้าง public-private key pair

In [ ]:
openssl genrsa -out mr_b.key 512

In [ ]:
ls mr_b.key

Mr.B สร้าง public key เพื่อส่งให้ Mr.A ใช้ encrypt message

In [ ]:
openssl rsa -in mr_b.key -pubout -out mr_b.pub.key.pem

In [ ]:
ls mr_b.pub.key.pem

### Mr.A

Mr.A ต้องการส่ง message
```
"Hello B, I'am A."
```
ไปหา Mr.B

In [ ]:
echo "Hello B, I'am A." > a2b_message.txt

Mr.A encrypt message เพื่อทำการส่งให้ Mr.B

In [ ]:
openssl rsautl -encrypt -pubin -inkey mr_b.pub.key.pem -in a2b_message.txt -out encrypted_a2b_message.txt

ข้อมูลที่ถูก encrypted message

In [ ]:
cat encrypted_a2b_message.txt

Mr.A สร้าง private key สำหรับการสร้าง public-private key pair

In [ ]:
openssl genrsa -out mr_a.key 512

In [ ]:
ls mr_a.key

Mr.A สร้าง public key เพื่อส่งให้ Mr.B validate signature ได้

In [ ]:
openssl rsa -in mr_a.key -pubout -out mr_a.pub.key.pem

In [ ]:
ls mr_a.pub.key.pem

Sign document เพื่อสร้าง digital signature

In [ ]:
openssl dgst -md5 -sign mr_a.key -out digital_signature.md5 encrypted_a2b_message.txt

digital signature

In [ ]:
cat digital_signature.md5

### Mr.B

รับ public key, digital signature และ encrypted message

Verify document ด้วย digital signature และ public key

In [ ]:
openssl dgst -md5 -verify mr_a.pub.key.pem -signature digital_signature.md5 encrypted_a2b_message.txt

Trying to decrypt message

In [ ]:
openssl rsautl -decrypt -inkey mr_b.key -in encrypted_a2b_message.txt -out decrypted_a2b_message.txt
cat decrypted_a2b_message.txt

## Conclusion

- ใช้ Public-Private Key 2 คู่ คู่หนึ่งใช้เพื่อ encrypt-decrypt message และอีกคู่หนึ่งใช้เพื่อ sign-verify message
- public key จะถูกใช้เพื่อ encrypt เสมอ เนื่อจาก แม้ว่า public key จะหลุดสู่สาธารณะ ก็สามารถใช้ทำได้เพียงเข้ารหัสเท่านั้น เนื่องจากคู่ private key จะอยู่ที่ receiver เสมอ
- private key ที่ถูกใช้ sign message ก็เช่นกัน โดย private key จะอยู่ที่ sender เสมอ และ public key ที่ถูกส่งให้ก็ทำได้เพียงใช้ verify ท่านั้น
- public key และ private key สามารถใช้ได้ทั้งเข้ารหัสและถอดรหัสได้ แต่ปกติ private key จะต้องไม่ถูกส่งผ่านตัวกลาง จะมีเฉพาะ public key เท่านั้นี่ถูกส่งผ่านตัวกลาง
- digital signature ก็ใช้ asymmetric key encryption เช่นเดียวกัน แต่ใช้เพื่อ verify เพื่อป้องกันการโจมตีแบบ man-in-the-middle attack โดยจะใช้เพื่อ validate message ว่าถูกส่งมาจาก sender จริงหรือไม่ (ใครก็สามารถใช้ public key ของ receiver เพื่อ encrypt ข้อมูลใดๆ ได้เช่นเดียวกัน)
- Public-Private Key ทั้ง 2 คู่ยังไม่มีความน่าเชื่อถือ เนื่องจากสามารถ generate ได้เองและไม่มีคนรับรองว่าถูก generate มาด้วย Mr.A / Mr.B จริงๆ
- ยังไม่สามารถ verify public key ได้ อาจจถูกสับเปลี่ยน public key ระหว่างทางได้

## References

* [Public-Private-Key Encryption and Digital Signature](https://www.researchgate.net/figure/Public-Private-Key-Encryption-and-Digital-Signatures_fig31_277736892)
* [SSL Client Authentication step-by-step](https://www.makethenmakeinstall.com/2014/05/ssl-client-authentication-step-by-step/)